# Module import

In [55]:
import pandas as pd
import numpy as np
import cv2
from glob import glob
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as tf
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image

#Import data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/My Drive/Github/COSE474_Term_Project
!ls

/content/drive/My Drive/Github/COSE474_Term_Project
dataset  dataset.zip  README.md  SDXL_Detector.ipynb


In [4]:
#!unzip -qq dataset.zip -d "/content/drive/My Drive/Github/COSE474_Term_Project/dataset"

#from glob import glob

#for purpose in ["train", "test"]:
#  for label in ["FAKE", "REAL"]:
#    filepaths = list(glob('/content/drive/MyDrive/Github/COSE474_Term_Project/dataset/{purpose}/{label}/*.jpg'.format(purpose=purpose, label=label)))
#    print(len(filepaths))

#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')

## Prepare train/test data

In [14]:
TRAIN_ANNO_PATH = "./dataset/train_labels.csv"

In [5]:
!ls

dataset  dataset.zip  README.md  SDXL_Detector.ipynb


In [8]:
#train_paths_fake = list(glob('./dataset/train/FAKE/*'))
#print(len(train_paths_fake))

#df_train_fake = pd.DataFrame(train_paths_fake)
#df_train_fake.columns = ['path']
#df_train_fake['label'] = 0
#df_train_fake.head()

#train_paths_real = list(glob('./dataset/train/REAL/*'))
#print(len(train_paths_real))

#df_train_real = pd.DataFrame(train_paths_real)
#df_train_real.columns = ['path']
#df_train_real['label'] = 1
#df_train_real.head()

#df_train = pd.concat((df_train_fake, df_train_real), axis = 0 )
#print(df_train.shape)

#df_train.to_csv("./dataset/train_labels.csv")

50000


,path,label
0,./dataset/train/FAKE/5929 (9).jpg,0
1,./dataset/train/FAKE/5865 (8).jpg,0
2,./dataset/train/FAKE/5850.jpg,0
3,./dataset/train/FAKE/5929 (5).jpg,0
4,./dataset/train/FAKE/5990 (2).jpg,0


In [29]:
df = pd.read_csv(TRAIN_ANNO_PATH)
df.drop(columns = ["Unnamed: 0"]).to_csv(TRAIN_ANNO_PATH, index=False)

In [30]:
df = pd.read_csv(TRAIN_ANNO_PATH)

In [38]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, target_transform=None):
      self.img_labels = dataframe
      self.img_dir = img_dir
      self.transform = transform
      self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [62]:
training_data = CustomImageDataset(pd.read_csv(TRAIN_ANNO_PATH), "", transform=lambda img: img.type(torch.FloatTensor))
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

## Model

In [47]:
NUM_EPOCHS = 1

In [84]:
# Check for GPU availability
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the Sequential model
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),
    nn.Flatten(),
    nn.Linear(32 * 8 * 8, 1),
    nn.Sigmoid(),
)

# Initialize the weights of the model
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

model.apply(weights_init)
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



### Sanity check

In [85]:
# Data loading check
sample_loader = DataLoader(training_data, batch_size=5, shuffle=True)
sample_batch = next(iter(sample_loader))
sample_images, sample_labels = sample_batch

# Verify data loading
print("Sample images shape:", sample_images.shape)
print("Sample labels:", sample_labels)


Sample images shape: torch.Size([5, 3, 32, 32])
Sample labels: tensor([0, 1, 1, 0, 0])


In [88]:
losses = []

# Assuming train_loader provides batches of input data and corresponding sample_labels
for epoch in range(100):
    epoch_loss = 0.0  # Initialize epoch loss

    # Move sample_images and labels to the GPU
    inputs, labels = sample_images.to(device), sample_labels.to(device)

    optimizer.zero_grad()  # Zero the gradients
    outputs = model(inputs)  # Forward pass

    loss = criterion(outputs, labels)  # Calculate loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights based on gradients

    # Accumulate epoch loss
    epoch_loss += loss.item()

    # Optionally, print the loss

    # Calculate average epoch loss and store it
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    losses.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Average Epoch Loss: {avg_epoch_loss:.4f}")


RuntimeError: ignored

In [75]:
losses = []

# Assuming train_loader provides batches of input data and corresponding labels
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0  # Initialize epoch loss
    for i, data in enumerate(train_dataloader, start=0):
        inputs, labels = data  # Get inputs and corresponding labels

        # Move inputs and labels to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights based on gradients

        # Accumulate epoch loss
        epoch_loss += loss.item()

        # Optionally, print the loss
        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")

    # Calculate average epoch loss and store it
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    losses.append(avg_epoch_loss)
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Average Epoch Loss: {avg_epoch_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt

# Plotting the loss
plt.plot(range(1, NUM_EPOCHS + 1), losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()
